In [1]:
import os
import yaml
import time
import datetime
import warnings
import sys
sys.path.append("../scr")
from tqdm import tqdm

import torch
import timm
import pandas as pd
import numpy as np
import torch.nn as nn

from box import Box
from torch.utils.data import DataLoader
from loguru import logger

from utils.create_dataset import BirdDataset
from utils.base_utils import set_seed
from utils.metrics import validation_epoch_end




In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
date_now = datetime.datetime.now().strftime("%d_%B_%Y_%H_%M")

path_save = os.path.join("../experiment", date_now)
if not os.path.exists(path_save):
    os.makedirs(path_save)

# Load config with params for training
with open("../scr/config_tf.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    config = Box(config)

logger.add(f"{path_save}/info_log{date_now}.log",
           format="<red>{time:YYYY-MM-DD HH:mm:ss}</red>| {message}")


logger.info(f"Folder with experiment- {path_save}")
logger.info("----------params----------")

for param in config:
    logger.info(f"{param}: {str(config[param])}")

# Create device for training and set_seed:
config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
set_seed(seed=config.seed)

# Read data
df = pd.read_csv("../data/data.csv")
df_train, df_test = (df[df.fold != 3].reset_index(drop=True),
                     df[df.fold == 3].reset_index(drop=True)
                     )
logger.info(f"Size df_train- {df_train.shape[0]}")
logger.info(f"Size df_test- {df_test.shape[0]}")


2023-10-05 15:54:59.370 | INFO     | __main__:<module>:17 - Folder with experiment- ../experiment\05_October_2023_15_54
2023-10-05 15:54:59.371 | INFO     | __main__:<module>:18 - ----------params----------
2023-10-05 15:54:59.373 | INFO     | __main__:<module>:21 - debug: False
2023-10-05 15:54:59.374 | INFO     | __main__:<module>:21 - seed: 1771
2023-10-05 15:54:59.375 | INFO     | __main__:<module>:21 - path_to_files_base: ../data
2023-10-05 15:54:59.377 | INFO     | __main__:<module>:21 - batch_size: 16
2023-10-05 15:54:59.378 | INFO     | __main__:<module>:21 - num_workers: 0
2023-10-05 15:54:59.379 | INFO     | __main__:<module>:21 - optimizer_lr: 0.006
2023-10-05 15:54:59.380 | INFO     | __main__:<module>:21 - optimizer_wd: 0
2023-10-05 15:54:59.381 | INFO     | __main__:<module>:21 - model_name: tf_efficientnet_b0
2023-10-05 15:54:59.382 | INFO     | __main__:<module>:21 - metric: custom
2023-10-05 15:54:59.383 | INFO     | __main__:<module>:21 - epochs: 4
2023-10-05 15:54:59

In [3]:

dataset_train = BirdDataset(df=df_train,
                            path_to_folder_with_audio=config.path_to_files_base,
                            tensorflow=True
                            )
dataset_test = BirdDataset(df=df_test,
                           path_to_folder_with_audio=config.path_to_files_base,
                           tensorflow=True
                           )

train_loader = DataLoader(dataset_train,
                          batch_size=config.batch_size,
                          shuffle=True,
                          num_workers=config.num_workers)
valid_loader = DataLoader(dataset_test,
                          batch_size=config.batch_size,
                          num_workers=config.num_workers)

In [4]:
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf

In [5]:
model = tf.keras.applications.efficientnet.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(626, 256, 3),
    drop_connect_rate=0.4,
)

input_shape = (626, 256, 3)
# Define the pre-processing layers for the input spectrogram
# preprocessing_layers = [
#     preprocessing.Normalization(),
# ]

# Define the spectrogram classification model
model = tf.keras.Sequential([
    tf.keras.Input(shape=input_shape),
    # *preprocessing_layers,
    model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(264, activation='sigmoid')])


In [6]:
learning_rate_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=config.optimizer_lr,
    decay_steps=config.epochs * len(train_loader),  # Устанавливаем decay_steps равным общему числу эпох
    alpha=0.0  # alpha устанавливается в 0.0 для чистой косинусной функции
)

In [7]:
loss_fn = tf.keras.losses.BinaryCrossentropy()  # Для бинарной классификации
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

In [8]:
# Получите текущее значение learning rate
current_learning_rate = optimizer.learning_rate.numpy()

In [9]:
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [10]:
model = model
metric = validation_epoch_end

In [11]:
logger.info(f"Starting train. Model - {config.model_name}")
for epoch_i in range(1, config.epochs + 1):
    if config.debug:
        k = 1
    start = time.time()
    logger.info(f'---------------------epoch:{epoch_i}/{config.epochs}---------------------')

    # loss
    avg_train_loss = 0
    avg_val_loss = 0
    predicted_labels_list = None
    true_labels_list = None

    ############## Train #############
    train_pbar = tqdm(train_loader, desc="Training")
    for batch in train_pbar:
        X_batch = batch[0]
        y_batch = batch[1]

        #convert to tf 
        X_batch = tf.convert_to_tensor(X_batch.numpy(), dtype=tf.float32)
        y_batch = tf.convert_to_tensor(y_batch.numpy(), dtype=tf.float32)

        
        with tf.GradientTape() as tape:
            logits = model(X_batch, training=True)
            loss_value = loss_fn(y_batch, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        avg_train_loss += loss_value.numpy()

        if config.debug:
            k += 1
            if k > 3: break

    valid_pbar = tqdm(valid_loader, desc="Validation")
    for batch in (valid_pbar):
        X_batch = batch[0]
        y_b = batch[1]

        #convert to tf 
        X_batch = tf.convert_to_tensor(X_batch.numpy(), dtype=tf.float32)
        y_batch = tf.convert_to_tensor(y_b.numpy(), dtype=tf.float32)

        logits = model(X_batch, training=True)
        loss_value = loss_fn(y_batch, logits)
        if predicted_labels_list is None:
                predicted_labels_list = logits
                true_labels_list = y_b
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, logits.numpy()], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_b], axis=0)
            
        if config.debug:
            k += 1
            if k > 6:  break


    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0
    avg_metric = metric(all_true_labels, all_predicted_labels)

    logger.info(f'epoch: {epoch_i}')

    logger.info("loss_train: %0.4f| loss_valid: %0.4f|" % (avg_train_loss, avg_val_loss))
    for m in avg_metric:
        logger.info(f"metric {m} : {avg_metric[m]:.<5g}")

    elapsed_time = time.time() - start
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)
    logger.info(f"Elapsed time: {hours:02d}:{minutes:02d}:{seconds:02d}")
    tf.keras.saving.save_model(model, f'{path_save}/model_{config.model_name}_ep_{epoch_i}')
    

2023-10-05 15:55:08.779 | INFO     | __main__:<module>:1 - Starting train. Model - tf_efficientnet_b0
2023-10-05 15:55:08.782 | INFO     | __main__:<module>:6 - ---------------------epoch:1/4---------------------
Validation: 100%|██████████| 193/193 [11:40<00:00,  3.63s/it]
2023-10-05 18:35:14.065 | INFO     | __main__:<module>:67 - epoch: 1
2023-10-05 18:35:14.108 | INFO     | __main__:<module>:69 - loss_train: 18.7400| loss_valid: 0.0000|
2023-10-05 18:35:14.109 | INFO     | __main__:<module>:71 - metric val_RMAP : 0.107474
2023-10-05 18:35:14.111 | INFO     | __main__:<module>:77 - Elapsed time: 02:40:05


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_1\assets


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_1\assets
2023-10-05 18:35:39.957 | INFO     | __main__:<module>:6 - ---------------------epoch:2/4---------------------
Validation: 100%|██████████| 193/193 [11:31<00:00,  3.59s/it]
2023-10-05 21:13:36.886 | INFO     | __main__:<module>:67 - epoch: 2
2023-10-05 21:13:36.900 | INFO     | __main__:<module>:69 - loss_train: 13.9301| loss_valid: 0.0000|
2023-10-05 21:13:36.901 | INFO     | __main__:<module>:71 - metric val_RMAP : 0.157993
2023-10-05 21:13:36.902 | INFO     | __main__:<module>:77 - Elapsed time: 02:37:56


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_2\assets


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_2\assets
2023-10-05 21:14:01.236 | INFO     | __main__:<module>:6 - ---------------------epoch:3/4---------------------
Validation: 100%|██████████| 193/193 [13:54<00:00,  4.32s/it]
2023-10-05 23:52:10.404 | INFO     | __main__:<module>:67 - epoch: 3
2023-10-05 23:52:10.445 | INFO     | __main__:<module>:69 - loss_train: 11.4857| loss_valid: 0.0000|
2023-10-05 23:52:10.446 | INFO     | __main__:<module>:71 - metric val_RMAP : 0.190731
2023-10-05 23:52:10.448 | INFO     | __main__:<module>:77 - Elapsed time: 02:38:09


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_3\assets


INFO:tensorflow:Assets written to: ../experiment\05_October_2023_15_54/model_tf_efficientnet_b0_ep_3\assets
2023-10-05 23:52:44.452 | INFO     | __main__:<module>:6 - ---------------------epoch:4/4---------------------
Training:  32%|███▏      | 246/771 [56:43<2:01:04, 13.84s/it]


KeyboardInterrupt: 